# IOPS AI Challenge Dataset

In [10]:
import pandas as pd
import os
from typing import Final
from collections.abc import Callable
import matplotlib.pyplot as plt
from config import data_raw_folder, data_processed_folder
from timeeval import Datasets

In [11]:
plt.rcParams['figure.figsize'] = (20, 10)

In [12]:
dataset_collection_name = "IOPS"
source_folder = os.path.join(data_raw_folder, "IOPS AI Challenge/KPI Anomaly Detection Final Data Set")
target_folder = data_processed_folder

from pathlib import Path
print(f"Looking for source datasets in {Path(source_folder).absolute()} and\nsaving processed datasets in {Path(target_folder).absolute()}")

Looking for source datasets in /home/projects/akita/data/benchmark-data/data-raw/IOPS AI Challenge/KPI Anomaly Detection Final Data Set and
saving processed datasets in /home/projects/akita/data/benchmark-data/data-processed


In [13]:
def transform(df: pd.DataFrame, kpi: str) -> pd.DataFrame:
    df = df[df["KPI ID"] == kpi].copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df[["timestamp", "value", "label"]]
    df.columns = ["timestamp", "value", "is_anomaly"]
    return df

## Transformation

In [14]:
train_type = "supervised"
train_is_normal = False
input_type = "univariate"
datetime_index = True
dataset_type = "real"

# create target directory
dataset_subfolder = os.path.join(input_type, dataset_collection_name)
target_subfolder = os.path.join(target_folder, dataset_subfolder)
try:
    os.makedirs(target_subfolder)
    print(f"Created directories {target_subfolder}")
except FileExistsError:
    print(f"Directories {target_subfolder} already exist")
    pass

dm = Datasets(target_folder)

Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/IOPS already exist


In [15]:
print("Reading source datasets...")
df_train = pd.read_csv(os.path.join(source_folder, "phase2_train.csv"))
df_test = pd.read_hdf(os.path.join(source_folder, "phase2_ground_truth.hdf"))
df_test["KPI ID"] = df_test["KPI ID"].transform(lambda x: str(x))
dfs = {
    "train": df_train,
    "test": df_test
}
print("...done.\n\nStarting processing...")

for kpi in df_test["KPI ID"].unique():
    dataset_length = 0
    paths = {}

    # transform file
    for t_type in ["train", "test"]:
        filename = f"KPI-{kpi}.{t_type}.csv"
        path = os.path.join(dataset_subfolder, filename)
        target_filepath = os.path.join(target_subfolder, filename)
        paths[t_type] = path
        df_target = transform(dfs[t_type], kpi)
        if t_type == "test":
            dataset_length = len(df_target)
        df_target.to_csv(target_filepath, index=False)
        print(f"Processed KPI {kpi} -> {target_filepath}")

    dataset_name = kpi

    # save metadata
    dm.add_dataset((dataset_collection_name, dataset_name),
        train_path = paths["train"],
        test_path = paths["test"],
        dataset_type = dataset_type,
        datetime_index = datetime_index,
        split_at = None,
        train_type = train_type,
        train_is_normal = train_is_normal,
        input_type = input_type,
        dataset_length = dataset_length
    )
print("...done.")
dm.save()
print("Metadata saved.")

Reading source datasets...
...done.

Starting processing...
Processed KPI da10a69f-d836-3baa-ad40-3e548ecf1fbd -> /home/projects/akita/data/benchmark-data/data-processed/univariate/IOPS/KPI-da10a69f-d836-3baa-ad40-3e548ecf1fbd.train.csv
Processed KPI da10a69f-d836-3baa-ad40-3e548ecf1fbd -> /home/projects/akita/data/benchmark-data/data-processed/univariate/IOPS/KPI-da10a69f-d836-3baa-ad40-3e548ecf1fbd.test.csv
Processed KPI e0747cad-8dc8-38a9-a9ab-855b61f5551d -> /home/projects/akita/data/benchmark-data/data-processed/univariate/IOPS/KPI-e0747cad-8dc8-38a9-a9ab-855b61f5551d.train.csv
Processed KPI e0747cad-8dc8-38a9-a9ab-855b61f5551d -> /home/projects/akita/data/benchmark-data/data-processed/univariate/IOPS/KPI-e0747cad-8dc8-38a9-a9ab-855b61f5551d.test.csv
Processed KPI ab216663-dcc2-3a24-b1ee-2c3e550e06c9 -> /home/projects/akita/data/benchmark-data/data-processed/univariate/IOPS/KPI-ab216663-dcc2-3a24-b1ee-2c3e550e06c9.train.csv
Processed KPI ab216663-dcc2-3a24-b1ee-2c3e550e06c9 -> /ho

In [17]:
dm.refresh()
dm.df().loc[(slice(dataset_collection_name,dataset_collection_name), slice(None))]

train_path  \
collection_name dataset_name                                                                              
IOPS            05f10d3a-239c-3bef-9bdc-a2feeb0037aa  univariate/IOPS/KPI-05f10d3a-239c-3bef-9bdc-a2...   
                0efb375b-b902-3661-ab23-9a0bb799f4e3  univariate/IOPS/KPI-0efb375b-b902-3661-ab23-9a...   
                1c6d7a26-1f1a-3321-bb4d-7a9d969ec8f0  univariate/IOPS/KPI-1c6d7a26-1f1a-3321-bb4d-7a...   
                301c70d8-1630-35ac-8f96-bc1b6f4359ea  univariate/IOPS/KPI-301c70d8-1630-35ac-8f96-bc...   
                42d6616d-c9c5-370a-a8ba-17ead74f3114  univariate/IOPS/KPI-42d6616d-c9c5-370a-a8ba-17...   
                43115f2a-baeb-3b01-96f7-4ea14188343c  univariate/IOPS/KPI-43115f2a-baeb-3b01-96f7-4e...   
                431a8542-c468-3988-a508-3afd06a218da  univariate/IOPS/KPI-431a8542-c468-3988-a508-3a...   
                4d2af31a-9916-3d9f-8a8e-8a268a48c095  univariate/IOPS/KPI-4d2af31a-9916-3d9f-8a8e-8a...   
                54350a12-7a9d-3ca8-b81f-f886b9d156fd  univariate/IOPS/KPI-54350a12-7a9d-3ca8-b81f-f8...   
                55f8b8b8-b659-38df-b3df-e4a5a8a54bc9  univariate/IOPS/KPI-55f8b8b8-b659-38df-b3df-e4...   
                57051487-3a40-3828-9084-a12f7f23ee38  univariate/IOPS/KPI-57051487-3a40-3828-9084-a1...   
                6a757df4-95e5-3357-8406-165e2bd49360  univariate/IOPS/KPI-6a757df4-95e5-3357-8406-16...   
                6d1114ae-be04-3c46-b5aa-be1a003a57cd  univariate/IOPS/KPI-6d1114ae-be04-3c46-b5aa-be...   
                6efa3a07-4544-34a0-b921-a155bd1a05e8  univariate/IOPS/KPI-6efa3a07-4544-34a0-b921-a1...   
                7103fa0f-cac4-314f-addc-866190247439  univariate/IOPS/KPI-7103fa0f-cac4-314f-addc-86...   
                847e8ecc-f8d2-3a93-9107-f367a0aab37d  univariate/IOPS/KPI-847e8ecc-f8d2-3a93-9107-f3...   
                8723f0fb-eaef-32e6-b372-6034c9c04b80  univariate/IOPS/KPI-8723f0fb-eaef-32e6-b372-60...   
                9c639a46-34c8-39bc-aaf0-9144b37adfc8  univariate/IOPS/KPI-9c639a46-34c8-39bc-aaf0-91...   
                a07ac296-de40-3a7c-8df3-91f642cc14d0  univariate/IOPS/KPI-a07ac296-de40-3a7c-8df3-91...   
                a8c06b47-cc41-3738-9110-12df0ee4c721  univariate/IOPS/KPI-a8c06b47-cc41-3738-9110-12...   
                ab216663-dcc2-3a24-b1ee-2c3e550e06c9  univariate/IOPS/KPI-ab216663-dcc2-3a24-b1ee-2c...   
                adb2fde9-8589-3f5b-a410-5fe14386c7af  univariate/IOPS/KPI-adb2fde9-8589-3f5b-a410-5f...   
                ba5f3328-9f3f-3ff5-a683-84437d16d554  univariate/IOPS/KPI-ba5f3328-9f3f-3ff5-a683-84...   
                c02607e8-7399-3dde-9d28-8a8da5e5d251  univariate/IOPS/KPI-c02607e8-7399-3dde-9d28-8a...   
                c69a50cf-ee03-3bd7-831e-407d36c7ee91  univariate/IOPS/KPI-c69a50cf-ee03-3bd7-831e-40...   
                da10a69f-d836-3baa-ad40-3e548ecf1fbd  univariate/IOPS/KPI-da10a69f-d836-3baa-ad40-3e...   
                e0747cad-8dc8-38a9-a9ab-855b61f5551d  univariate/IOPS/KPI-e0747cad-8dc8-38a9-a9ab-85...   
                f0932edd-6400-3e63-9559-0a9860a1baa9  univariate/IOPS/KPI-f0932edd-6400-3e63-9559-0a...   
                ffb82d38-5f00-37db-abc0-5d2e4e4cb6aa  univariate/IOPS/KPI-ffb82d38-5f00-37db-abc0-5d...   

                                                                                              test_path  \
collection_name dataset_name                                                                              
IOPS            05f10d3a-239c-3bef-9bdc-a2feeb0037aa  univariate/IOPS/KPI-05f10d3a-239c-3bef-9bdc-a2...   
                0efb375b-b902-3661-ab23-9a0bb799f4e3  univariate/IOPS/KPI-0efb375b-b902-3661-ab23-9a...   
                1c6d7a26-1f1a-3321-bb4d-7a9d969ec8f0  univariate/IOPS/KPI-1c6d7a26-1f1a-3321-bb4d-7a...   
                301c70d8-1630-35ac-8f96-bc1b6f4359ea  univariate/IOPS/KPI-301c70d8-1630-35ac-8f96-bc...   
                42d6616d-c9c5-370a-a8ba-17ead74f3114  univariate/IOPS/KPI-42d6616d-c9c5-370a-a8ba-17...   
                43115f2a-b

## Experimentation

In [ ]:
os.listdir(source_folder)

In [ ]:
train = pd.read_csv("data-raw/IOPS AI Challenge/KPI Anomaly Detection Final Data Set/phase2_train.csv")

In [ ]:
test = pd.read_hdf("data-raw/IOPS AI Challenge/KPI Anomaly Detection Final Data Set/phase2_ground_truth.hdf")

In [ ]:
test["KPI ID"] = test["KPI ID"].transform(lambda x: str(x))
test

In [ ]:
test["KPI ID"].unique()

In [ ]:
transform(train, "da10a69f-d836-3baa-ad40-3e548ecf1fbd")

In [ ]:
transform(test, "da10a69f-d836-3baa-ad40-3e548ecf1fbd")

In [ ]:
train_dfs = [x for _, x in train.groupby("KPI ID")]
train_dfs[0]

In [ ]:
test_dfs = [x for _, x in test.groupby("KPI ID")]
test_dfs[0]

In [ ]:
train_kpis = train["KPI ID"].unique()
test_kpis = test["KPI ID"].unique()
set(test) - set(train)

In [ ]:
for df in test_dfs:
    df.plot(x="timestamp", y=["value"])
plt.show()